## Get transcripts from videos with caption IDs

In [ ]:
import os
import json
from dotenv import load_dotenv
import pandas as pd
from google.auth.transport.requests import Request as AuthRequest
from googleapiclient.discovery import build
import httplib2
from pprint import pprint
from datetime import timedelta
import re
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Access API key
yt_api_key = os.getenv("YOUTUBE_API_KEY")

In [ ]:
# Create a YouTube API client
youtube = build('youtube', 'v3', developerKey=yt_api_key)

In [ ]:
# Load videos list


In [ ]:
# Function to download and parse caption file to extract transcript text
def download_and_parse_caption(caption_id):
    # Download the caption file
    request = youtube.captions().download(
        id=caption_id,
        tfmt="srt"  # Choose the caption format (e.g., "srt", "vtt")
    )
    caption_response = request.execute()

    # Parse the caption file to extract transcript text
    transcript_text = caption_response.decode('utf-8')  # Assuming the response is in UTF-8 encoding

    return transcript_text

In [ ]:
### MAIN ###

for caption_id in 
captions_data = get_captions(video_id)

# Create a directory to save transcripts
if not os.path.exists("transcripts"):
    os.makedirs("transcripts")

# Download and save transcripts for each caption
for caption in captions_data:
    caption_id = caption['id']
    transcript_text = download_and_parse_caption(caption_id)

    # Save transcript text to a file
    file_name = f"transcripts/{video_id}_{caption_id}.txt"
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(transcript_text)

    print(f"Transcript saved to {file_name}")